# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Нам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Необходимо построить модель с предельно большим значением *F1*-меры. Чтобы выполнить задачу успешно, нужно довести метрику до 0.59.

Дополнительно измерим *AUC-ROC*, сравнивним её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

### Обзор данных

* По целевому признаку можем определить, что предстоить решить задачу бинарной классификации (1 - клиент ушел из банка, 0 - остался).
* Заголовки написаны в верблюжьем стиле, следует изменить на змеиный. Также, в названиях присутствуют глаголы.
* В столбце 'Tenure' есть пропуски, и у столбца тип 'float', хотя все признаки целочисленные.
* Присутствуют категориальные признаки, следовательно для улучшения показателей модели необходима кодировка и последующее масштабирование переменных. Кодирование будем производить техникой OHE, так как она работает со всеми моделями.
* В датафрейме по условию имеется индексация объектов, исходя из этого столбец 'RowNumber' теряет свой смысл, его можно удалить.

In [2]:
# импортирование всех нужных библиотек и фуцнкций 
import pandas as pd
import numpy as np
import re
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.utils import shuffle
from sklearn.impute import KNNImputer

# формирование датафрейма 
data = pd.read_csv('dataset_churn.csv', index_col=0)

# просмотр общей информации о датафрейме
display(data.head())
data.info()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


### Стиль заголовков

Приведение заголовков к змеиному регистру.

In [224]:
# изменение стиля заголовков
for name in data.columns:
    rena = re.sub(r"(?=[A-Z])(?!^)", '_', name).lower()
    data = data.rename(columns={name:rena})

# проверка внесенных изменений
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_number        10000 non-null  int64  
 1   customer_id       10000 non-null  int64  
 2   surname           10000 non-null  object 
 3   credit_score      10000 non-null  int64  
 4   geography         10000 non-null  object 
 5   gender            10000 non-null  object 
 6   age               10000 non-null  int64  
 7   tenure            9091 non-null   float64
 8   balance           10000 non-null  float64
 9   num_of_products   10000 non-null  int64  
 10  has_cr_card       10000 non-null  int64  
 11  is_active_member  10000 non-null  int64  
 12  estimated_salary  10000 non-null  float64
 13  exited            10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


### Дубликаты

Дубликатов не выявлено.

In [225]:
# проверка данных на наличие дубликатов
display(data.duplicated().sum())
display(data['customer_id'].duplicated().sum())

0

0

### Пропуски

Пропуски в столбце 'tenure' можно заполнить используя метод k-ближайших соседей.

In [226]:
# выделение количественных признаков
numeric = ['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary']

# создание KNNImputer объекта и его обучение
imputer = KNNImputer(n_neighbors=2)
imputer.fit(data[numeric])

# заполнение пропусков
data['tenure'] = pd.DataFrame(data=imputer.transform(data[numeric]), columns=data[numeric].columns)['tenure']

# проверка внесенных изменений
data.info()
display(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_number        10000 non-null  int64  
 1   customer_id       10000 non-null  int64  
 2   surname           10000 non-null  object 
 3   credit_score      10000 non-null  int64  
 4   geography         10000 non-null  object 
 5   gender            10000 non-null  object 
 6   age               10000 non-null  int64  
 7   tenure            10000 non-null  float64
 8   balance           10000 non-null  float64
 9   num_of_products   10000 non-null  int64  
 10  has_cr_card       10000 non-null  int64  
 11  is_active_member  10000 non-null  int64  
 12  estimated_salary  10000 non-null  float64
 13  exited            10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


### Исправление ошибок

Удаление столбца 'row_number'

In [227]:
# удаление лишнего столбца
data = data.drop(['row_number'], axis=1)

# проверка внесенных изменений
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       10000 non-null  int64  
 1   surname           10000 non-null  object 
 2   credit_score      10000 non-null  int64  
 3   geography         10000 non-null  object 
 4   gender            10000 non-null  object 
 5   age               10000 non-null  int64  
 6   tenure            10000 non-null  float64
 7   balance           10000 non-null  float64
 8   num_of_products   10000 non-null  int64  
 9   has_cr_card       10000 non-null  int64  
 10  is_active_member  10000 non-null  int64  
 11  estimated_salary  10000 non-null  float64
 12  exited            10000 non-null  int64  
dtypes: float64(3), int64(7), object(3)
memory usage: 1015.8+ KB


### Подготовка признаков

#### Кодирование OHE

Признаки 'surname' и 'customer_id' не влияют на вероятность ухода клиента из банка, поэтому их стоит отбросить до обучения модели.

In [228]:
# формирование датафрейма без столбцов не влияюших на целевой признак
data_ohe = data.drop(['surname', 'customer_id'], axis=1)

# кодируем категориальные признаки, избегая дамми-ловушку
data_ohe = pd.get_dummies(data_ohe, drop_first=True)

# проверка внесенных изменений
data_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   credit_score       10000 non-null  int64  
 1   age                10000 non-null  int64  
 2   tenure             10000 non-null  float64
 3   balance            10000 non-null  float64
 4   num_of_products    10000 non-null  int64  
 5   has_cr_card        10000 non-null  int64  
 6   is_active_member   10000 non-null  int64  
 7   estimated_salary   10000 non-null  float64
 8   exited             10000 non-null  int64  
 9   geography_Germany  10000 non-null  uint8  
 10  geography_Spain    10000 non-null  uint8  
 11  gender_Male        10000 non-null  uint8  
dtypes: float64(3), int64(6), uint8(3)
memory usage: 732.5 KB


## Исследование задачи

### Баланс классов 

Присутствует дисбаланс классов, соотношение отрицательных ответов к положительным 4:1.

In [229]:
# соотношение значений целевого признака
display(data_ohe['exited'].value_counts())

0    7963
1    2037
Name: exited, dtype: int64

### Обучение моделей с учетом дисбаланса

Так как тестовой выборки нет, разобьем исходнные данные в соотношении 3:1:1. <br>
Также проведем масштабирование признаков путем стандартизации.

In [230]:
# формирование обучающей, валидационной и тестовой выборок
data_train, data_temporary = train_test_split(data_ohe, test_size=0.4, random_state=123, stratify=data_ohe['exited'])
data_valid, data_test = train_test_split(data_temporary, test_size=0.5, random_state=123, stratify=data_temporary['exited'])

# проверка соотношения размеров выборок
display(len(data_train), len(data_valid), len(data_test))

# применение стандартизации данных
scaler = StandardScaler()
data_train[numeric] = scaler.fit_transform(data_train[numeric])
data_valid[numeric] = scaler.fit_transform(data_valid[numeric])
data_test[numeric] = scaler.fit_transform(data_test[numeric])

# признаки и целевой признак в обучающей выборке 
features_train = data_train.drop(['exited'], axis=1)
target_train = data_train['exited']

# признаки и целевой признак в валидационной выборке 
features_valid = data_valid.drop(['exited'], axis=1)
target_valid = data_valid['exited']

# признаки и целевой признак в тестовой выборке 
features_test = data_test.drop(['exited'], axis=1)
target_test = data_test['exited']

6000

2000

2000

/tmp/ipykernel_24/1061406843.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train[numeric] = scaler.fit_transform(data_train[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/tmp/ipykernel_24/1061406843.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

#### Дерево решений 

* F1-мера: 0.53
* AUC-ROC: 0.78

In [231]:
# вариации гиперпараметров
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]

# сетка гиперпараметров
parameters_grid = {'min_samples_split':min_samples_split,
                   'min_samples_leaf':min_samples_leaf,
                   'max_depth':max_depth}

# модель 'дерево решений'
model_tree = GridSearchCV(DecisionTreeClassifier(random_state=123), parameters_grid, scoring='f1', cv=3)

# обучение модели
model_tree.fit(features_train, target_train)

# предсказание модели
predictions_valid = model_tree.predict(features_valid)

# вероятность значений целевого признака
probabilities_valid = model_tree.predict_proba(features_valid)

# вероятность положительного значения целевого признака
probabilities_one_valid = probabilities_valid[:, 1]

In [232]:
# подобранные гиперпараметры, показывающие наивысшую F1-меру
display(model_tree.best_params_)

# AUC-ROC значение модели
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

# метрики и матрица ошибок модели
display(f1_score(target_valid, predictions_valid))
display(confusion_matrix(target_valid, predictions_valid))
display(auc_roc)

{'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10}

0.5381294964028777

array([[1492,  100],
       [ 221,  187]])

0.788268086757316

#### Случайный лес

* F1-мера: 0.57
* AUC-ROC: 0.82

In [233]:
# вариации гиперпараметров
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
n_estimators = [int(x) for x in np.linspace(10, 30, num = 3)]

# сетка гиперпараметров
parameters_grid = {'min_samples_split':min_samples_split,
                   'min_samples_leaf':min_samples_leaf,
                   'max_depth':max_depth,
                   'n_estimators':n_estimators}

# модель 'случайный лес'
model_forest = GridSearchCV(RandomForestClassifier(random_state=123), parameters_grid, scoring='f1', cv=4)

# обучение модели
model_forest.fit(features_train, target_train)

# предсказание модели
predictions_valid = model_forest.predict(features_valid)

# вероятность значений целевого признака
probabilities_valid = model_forest.predict_proba(features_valid)

# вероятность положительного значения целевого признака
probabilities_one_valid = probabilities_valid[:, 1]

In [234]:
# подобранные гиперпараметры, показывающие наивысшую F1-меру
display(model_forest.best_params_)

# AUC-ROC значение модели
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

# метрики и матрица ошибок модели
display(f1_score(target_valid, predictions_valid))
display(confusion_matrix(target_valid, predictions_valid))
display(auc_roc)

{'max_depth': 20,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 20}

0.5718562874251496

array([[1523,   69],
       [ 217,  191]])

0.8294936693270274

#### Логистическая регрессия

* F1-мера: 0.28
* AUC-ROC: 0.73

In [235]:
# вариации гиперпараметров
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

# сетка гиперпараметров
parameters_grid = {'solver':solver}

# модель 'логистическая регрессия'
model_log_reg = GridSearchCV(LogisticRegression(random_state=123), parameters_grid, scoring='f1', cv=4)

# обучение модели
model_log_reg.fit(features_train, target_train)

# предсказание модели
predictions_valid = model_log_reg.predict(features_valid)

# вероятность значений целевого признака
probabilities_valid = model_log_reg.predict_proba(features_valid)

# вероятность положительного значения целевого признака
probabilities_one_valid = probabilities_valid[:, 1]

In [236]:
# подобранные гиперпараметры, показывающие наивысшую F1-меру
display(model_log_reg.best_params_)

# AUC-ROC значение модели
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

# метрики и матрица ошибок модели
display(f1_score(target_valid, predictions_valid))
display(confusion_matrix(target_valid, predictions_valid))
display(auc_roc)

{'solver': 'liblinear'}

0.2851919561243144

array([[1531,   61],
       [ 330,   78]])

0.7309202261306532

#### Константная модель 

Все три модели: "дерево решений", "случайный лес" и "логистическая регрессия" показывают значения метрик лучше чем константная модель при обучении и предсказывании без учета дисбаланса классов. У модели "случайный лес" самые высокие значения среднего гармонического полноты и точности (F1-мера), а также площади кривой соотношения TPR к FPR (AUC-ROC).

In [237]:
# константная модель
model_const = DummyClassifier(random_state=123, strategy='most_frequent')

# обучение константной модели
model_const.fit(features_train, target_train)

# значение AUC-ROC константной модели
display(roc_auc_score(target_valid, model_const.predict_proba(features_valid)[:, 1]))

0.5

## Борьба с дисбалансом

### Upsampling

При обучении моделей с техникой работы с дисбалансом upsampling результаты лучше, чем при игнорировании дисбаланса, наилучший результат у модели 'случайный лес':

* F1-мера: 0.59
* AUC-ROC: 0.83

In [238]:
# функция по методу upsampling
def upsample(features, target, repeat):
    
    # выделение признаков и целевого признака по значению целевого признака
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    # соединение признаков 
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)    
    
    # перемешивание объектов
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=123)
    
    return features_upsampled, target_upsampled

# формирование признаков и целевого признака методом upsampling
features_upsampled, target_upsampled = upsample(features_train, target_train, repeat=3)

# соотношение значений целевого признака
display(len(features_upsampled[target_train==1]), len(features_upsampled[target_train==0]))

/tmp/ipykernel_24/2944113024.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  display(len(features_upsampled[target_train==1]), len(features_upsampled[target_train==0]))
/tmp/ipykernel_24/2944113024.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  display(len(features_upsampled[target_train==1]), len(features_upsampled[target_train==0]))


3666

4778

#### Случайный лес на измененных данных методом upsampling

* Порог вероятности: 0.42
* F1-мера: 0.59
* AUC-ROC: 0.83

In [239]:
# вариации гиперпараметров
class_weight = ['balanced']
min_samples_split = [2, 5]
min_samples_leaf = [1, 2]
max_depth = [int(x) for x in np.linspace(10, 30, num = 3)]
n_estimators = [int(x) for x in np.linspace(10, 30, num = 3)]

# сетка гиперпараметров
parameters_grid = {'min_samples_split':min_samples_split,
                   'min_samples_leaf':min_samples_leaf,
                   'max_depth':max_depth,
                   'n_estimators':n_estimators,
                   'class_weight':class_weight}

# модель 'случайный лес'
model_forest_upsample = GridSearchCV(RandomForestClassifier(random_state=123), parameters_grid, scoring='f1', cv=5)

# обучение модели на измененных данных
model_forest_upsample.fit(features_upsampled, target_upsampled)

# предсказание модели
predictions_valid = model_forest_upsample.predict(features_valid)

# вероятность значений целевого признака
probabilities_valid = model_forest_upsample.predict_proba(features_valid)

# вероятность положительного значения целевого признака
probabilities_one_valid = probabilities_valid[:, 1]

# нахождение порога вероятности с наилучшими метриками
max_f1_score = 0
best_threshold = 0
for threshold in np.arange(0, 0.8, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    f1 = f1_score(predicted_valid, target_valid)
    if max_f1_score < f1:
        max_f1_score = f1
        best_threshold = threshold

# подобранные гиперпараметры, показывающие наивысшую F1-меру и метрики модели
display('Случайный лес')
display(model_forest_upsample.best_params_)
display(best_threshold, max_f1_score)
display(roc_auc_score(target_valid, probabilities_one_valid))

'Случайный лес'

{'class_weight': 'balanced',
 'max_depth': 20,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 30}

0.42

0.5969447708578144

0.8363878214602423

#### Логистическая регрессия на измененных данных методом upsampling

* Порог вероятности: 0.54
* F1-мера: 0.46
* AUC-ROC: 0.73

In [240]:
# вариации гиперпараметров
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
class_weight = ['balanced']

# сетка гиперпараметров
parameters_grid = {'solver':solver, 
                   'class_weight':class_weight}

# модель 'логистическая регрессия'
model_log_reg_upsample = GridSearchCV(LogisticRegression(random_state=123), parameters_grid, scoring='f1', cv=4)

# обучение модели на измененных данных
model_log_reg_upsample.fit(features_upsampled, target_upsampled)

# предсказание модели
predictions_valid = model_log_reg_upsample.predict(features_valid)

# вероятность значений целевого признака
probabilities_valid = model_log_reg_upsample.predict_proba(features_valid)

# вероятность положительного значения целевого признака
probabilities_one_valid = probabilities_valid[:, 1]

# нахождение порога вероятности с наилучшими метриками
max_f1_score = 0
best_threshold = 0
for threshold in np.arange(0, 0.8, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    f1 = f1_score(predicted_valid, target_valid)
    if max_f1_score < f1:
        max_f1_score = f1
        best_threshold = threshold

# подобранные гиперпараметры, показывающие наивысшую F1-меру и метрики модели
display('Логистическая регрессия')
display(model_log_reg_upsample.best_params_)
display(best_threshold, max_f1_score)
display(roc_auc_score(target_valid, probabilities_one_valid))

'Логистическая регрессия'

{'class_weight': 'balanced', 'solver': 'liblinear'}

0.54

0.466472303206997

0.7339608582126318

#### Дерево решений на измененных данных методом upsampling

* Порог вероятности: 0.0
* F1-мера: 0.48
* AUC-ROC: 0.67

In [241]:
# вариации гиперпараметров
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
class_weight = ['balanced']

# сетка гиперпараметров
parameters_grid = {'min_samples_split':min_samples_split,
                   'min_samples_leaf':min_samples_leaf,
                   'max_depth':max_depth, 
                   'class_weight':class_weight}

# модель 'дерево решений'
model_tree_upsample = GridSearchCV(DecisionTreeClassifier(random_state=123), parameters_grid, scoring='f1', cv=3)

# обучение модели на измененных данных
model_tree_upsample.fit(features_upsampled, target_upsampled)

# предсказание модели
predictions_valid = model_tree_upsample.predict(features_valid)

# вероятность значений целевого признака
probabilities_valid = model_tree_upsample.predict_proba(features_valid)

# вероятность положительного значения целевого признака
probabilities_one_valid = probabilities_valid[:, 1]

# нахождение порога вероятности с наилучшими метриками
max_f1_score = 0
best_threshold = 0
for threshold in np.arange(0, 0.8, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    f1 = f1_score(predicted_valid, target_valid)
    if max_f1_score < f1:
        max_f1_score = f1
        best_threshold = threshold

# подобранные гиперпараметры, показывающие наивысшую F1-меру и метрики модели
display('Дерево решений')
display(model_tree_upsample.best_params_)
display(best_threshold, max_f1_score)
display(roc_auc_score(target_valid, probabilities_one_valid))

'Дерево решений'

{'class_weight': 'balanced',
 'max_depth': 30,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

0.0

0.4839506172839506

0.675497585969061

### Downsampling

Модель 'случайный лес' после обучения на данных к которым была применена техника downsampling имеет значения метрик чуть ниже, по сравнению с моделью, которая обучалась на данных, измененных методом upsampling. Модель 'логистическая регрессия' имеет показатели метрик такие же, какие были у этой же модели с техникой upsampling. У модели 'дерево решений' мтерики выше, чем у этой же модели с данными, измененными методом upsampling.

In [242]:
# функция по методу downsampling
def downsample(features, target, fraction):
    
    # выделение признаков и целевого признака по значению целевого признака
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    # соединение признаков 
    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=123)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=123)] + [target_ones])  
    
    # перемешивание объектов
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=123)
    
    return features_downsampled, target_downsampled

# формирование признаков и целевого признака методом downsampling
features_downsampled, target_downsampled = downsample(features_train, target_train, fraction=0.3)

# соотношение значений целевого признака
display(len(features_downsampled[target_train==1]), len(features_downsampled[target_train==0]))

/tmp/ipykernel_24/4238420971.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  display(len(features_downsampled[target_train==1]), len(features_downsampled[target_train==0]))
/tmp/ipykernel_24/4238420971.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  display(len(features_downsampled[target_train==1]), len(features_downsampled[target_train==0]))


1222

1433

#### Случайный лес на измененных данных методом downsampling

* Порог вероятности: 0.58
* F1-мера: 0.56
* AUC-ROC: 0.82

In [243]:
# вариации гиперпараметров
class_weight = ['balanced']
min_samples_split = [2, 5]
min_samples_leaf = [1, 2]
max_depth = [int(x) for x in np.linspace(10, 30, num = 3)]
n_estimators = [int(x) for x in np.linspace(10, 30, num = 3)]

# сетка гиперпараметров
parameters_grid = {'min_samples_split':min_samples_split,
                   'min_samples_leaf':min_samples_leaf,
                   'max_depth':max_depth,
                   'n_estimators':n_estimators,
                   'class_weight':class_weight}

# модель 'случайный лес'
model_forest_downsample = GridSearchCV(RandomForestClassifier(random_state=123), parameters_grid, scoring='f1', cv=5)

# обучение модели на измененных данных
model_forest_downsample.fit(features_downsampled, target_downsampled)

# предсказание модели
predictions_valid = model_forest_downsample.predict(features_valid)

# вероятность значений целевого признака
probabilities_valid = model_forest_downsample.predict_proba(features_valid)

# вероятность положительного значения целевого признака
probabilities_one_valid = probabilities_valid[:, 1]

# нахождение порога вероятности с наилучшими метриками
max_f1_score = 0
best_threshold = 0
for threshold in np.arange(0, 0.8, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    f1 = f1_score(predicted_valid, target_valid)
    if max_f1_score < f1:
        max_f1_score = f1
        best_threshold = threshold

# подобранные гиперпараметры, показывающие наивысшую F1-меру и метрики модели
display('Случайный лес')
display(model_forest_downsample.best_params_)
display(best_threshold, max_f1_score)
display(roc_auc_score(target_valid, probabilities_one_valid))

'Случайный лес'

{'class_weight': 'balanced',
 'max_depth': 10,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 20}

0.58

0.564042303172738

0.8267070647354416

#### Логистическая регрессия на измененных данных методом downsampling

* Порог вероятности: 0.52
* F1-мера: 0.46
* AUC-ROC: 0.73

In [244]:
# вариации гиперпараметров
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
class_weight = ['balanced']

# сетка гиперпараметров
parameters_grid = {'solver':solver, 
                   'class_weight':class_weight}

# модель 'логистическая регрессия'
model_log_reg_downsample = GridSearchCV(LogisticRegression(random_state=123), parameters_grid, scoring='f1', cv=4)

# обучение модели на измененных данных
model_log_reg_downsample.fit(features_downsampled, target_downsampled)

# предсказание модели
predictions_valid = model_log_reg_downsample.predict(features_valid)

# вероятность значений целевого признака
probabilities_valid = model_log_reg_downsample.predict_proba(features_valid)

# вероятность положительного значения целевого признака
probabilities_one_valid = probabilities_valid[:, 1]

# нахождение порога вероятности с наилучшими метриками
max_f1_score = 0
best_threshold = 0
for threshold in np.arange(0, 0.8, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    f1 = f1_score(predicted_valid, target_valid)
    if max_f1_score < f1:
        max_f1_score = f1
        best_threshold = threshold

# подобранные гиперпараметры, показывающие наивысшую F1-меру и метрики модели
display('Логистическая регрессия')
display(model_log_reg_downsample.best_params_)
display(best_threshold, max_f1_score)
display(roc_auc_score(target_valid, probabilities_one_valid))

'Логистическая регрессия'

{'class_weight': 'balanced', 'solver': 'newton-cg'}

0.52

0.4656771799628942

0.7337468592964824

#### Дерево решений на измененных данных методом downsampling

* Порог вероятности: 0.62
* F1-мера: 0.52
* AUC-ROC: 0.72

In [245]:
# вариации гиперпараметров
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
class_weight = ['balanced']

# сетка гиперпараметров
parameters_grid = {'min_samples_split':min_samples_split,
                   'min_samples_leaf':min_samples_leaf,
                   'max_depth':max_depth, 
                   'class_weight':class_weight}

# модель 'дерево решений'
model_tree_downsample = GridSearchCV(DecisionTreeClassifier(random_state=123), parameters_grid, scoring='f1', cv=3)

# обучение модели на измененных данных
model_tree_downsample.fit(features_downsampled, target_downsampled)

# предсказание модели
predictions_valid = model_tree_downsample.predict(features_valid)

# вероятность значений целевого признака
probabilities_valid = model_tree_downsample.predict_proba(features_valid)

# вероятность положительного значения целевого признака
probabilities_one_valid = probabilities_valid[:, 1]

# нахождение порога вероятности с наилучшими метриками
max_f1_score = 0
best_threshold = 0
for threshold in np.arange(0, 0.8, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    f1 = f1_score(predicted_valid, target_valid)
    if max_f1_score < f1:
        max_f1_score = f1
        best_threshold = threshold

# подобранные гиперпараметры, показывающие наивысшую F1-меру и метрики модели
display('Дерево решений')
display(model_tree_downsample.best_params_)
display(best_threshold, max_f1_score)
display(roc_auc_score(target_valid, probabilities_one_valid))

'Дерево решений'

{'class_weight': 'balanced',
 'max_depth': 10,
 'min_samples_leaf': 4,
 'min_samples_split': 2}

0.62

0.5204386839481555

0.7296200672480047

## Тестирование модели

По итогам тестирования наивысшие полученные метрики оказались у модели 'случайный лес', обученной на данных, к которым был применен метод upsampling:

* F1-мера: 0.62
* AUC-ROC: 0.85

### Модели обученные на данных с методом upsampling

In [246]:
# вероятность значений целевого признака
probabilities_test = model_forest_upsample.predict_proba(features_test)

# вероятность положительного значения целевого признака
probabilities_one_test = probabilities_test[:, 1]

# установка порога
predicted_test = probabilities_one_test > 0.42
        
# метрики модели с выбранным порогом
display('Случайный лес')
display(f1_score(predicted_test, target_test))
display(roc_auc_score(target_test, probabilities_one_test))

# вероятность значений целевого признака
probabilities_test = model_log_reg_upsample.predict_proba(features_test)

# вероятность положительного значения целевого признака
probabilities_one_test = probabilities_test[:, 1]

# установка порога
predicted_test = probabilities_one_test > .54

# метрики модели с выбранным порогом
display('Логистическая регрессия')
display(f1_score(predicted_test, target_test))
display(roc_auc_score(target_test, probabilities_one_test))

# вероятность значений целевого признака
probabilities_test = model_tree_upsample.predict_proba(features_test)

# вероятность положительного значения целевого признака
probabilities_one_test = probabilities_test[:, 1]

# установка порога
predicted_test = probabilities_one_test > .0

# метрики модели с выбранным порогом
display('Дерево решений')
display(f1_score(predicted_test, target_test))
display(roc_auc_score(target_test, probabilities_one_test))

'Случайный лес'

0.6229885057471264

0.8529376834461581

'Логистическая регрессия'

0.5152091254752852

0.7755413348633688

'Дерево решений'

0.5091352009744213

0.6924127517347856

### Модели обученные на данных с методом downsampling

In [247]:
# вероятность значений целевого признака
probabilities_test = model_forest_downsample.predict_proba(features_test)

# вероятность положительного значения целевого признака
probabilities_one_test = probabilities_test[:, 1]

# установка порога
predicted_test = probabilities_one_test > 0.42
        
# метрики модели с выбранным порогом
display('Случайный лес')
display(f1_score(predicted_test, target_test))
display(roc_auc_score(target_test, probabilities_one_test))

# вероятность значений целевого признака
probabilities_test = model_log_reg_downsample.predict_proba(features_test)

# вероятность положительного значения целевого признака
probabilities_one_test = probabilities_test[:, 1]

# установка порога
predicted_test = probabilities_one_test > .54

# метрики модели с выбранным порогом
display('Логистическая регрессия')
display(f1_score(predicted_test, target_test))
display(roc_auc_score(target_test, probabilities_one_test))

# вероятность значений целевого признака
probabilities_test = model_tree_downsample.predict_proba(features_test)

# вероятность положительного значения целевого признака
probabilities_one_test = probabilities_test[:, 1]

# установка порога
predicted_test = probabilities_one_test > .0

# метрики модели с выбранным порогом
display('Дерево решений')
display(f1_score(predicted_test, target_test))
display(roc_auc_score(target_test, probabilities_one_test))

'Случайный лес'

0.5575657894736842

0.854270294948261

'Логистическая регрессия'

0.5043144774688398

0.7754688432654534

'Дерево решений'

0.4107351225204201

0.765215138096494